In [1]:
import sqlite3
import pandas as pd

# %% [markdown]
# ## Setup dbase in RAM

In [2]:
con = sqlite3.connect(':memory:')
cur = con.cursor()

In [3]:
sql_dump = open(r"db_sql\db_dmp.sql","r")
sql_statements = (sql_dump
                    .read()
                    .split(";"))
sql_dump.close()
                
for st in sql_statements:
    try:
        cur.execute(st)
    except sqlite3.Error as e:
        print(e)              

# %% [markdown]
# ## Check DB (via list of tables)

In [4]:
pd.read_sql_query(
    """ SELECT name
        FROM sqlite_master 
        WHERE type ='table'
        AND name NOT LIKE 'sqlite_%'; """,
        con)

# %% [markdown]
# ## Run queries
# a. РќР°РїРёС€РёС‚Рµ Р·Р°РїСЂРѕСЃ, РєРѕС‚РѕСЂС‹Р№ РІС‹РІРѕРґРёС‚ РЅР°Р·РІР°РЅРёРµ РїРѕР·РёС†РёР№ (Р±РµР· РїРѕРІС‚РѕСЂРµРЅРёР№),
# С…РѕС‚СЏ Р±С‹ РѕРґРёРЅ РїСЂРµРґСЃС‚Р°РІРёС‚РµР»СЊ РєРѕС‚РѕСЂРѕР№ РїРѕР»СѓС‡Р°РµС‚ Р·Р°СЂРїР»Р°С‚Сѓ РІ РїСЂРѕРјРµР¶СѓС‚РєРµ 400-600

,name
0,medicine
1,diagnosis
2,job
3,workers
4,contacts


In [5]:
pd.read_sql_query(
    """ SELECT DISTINCT(POSITION)
        FROM job
        WHERE SALARY BETWEEN 400 AND 600; """,
        con)

# %% [markdown]
# b. РќР°РїРёС€РёС‚Рµ Р·Р°РїСЂРѕСЃ, РєРѕС‚РѕСЂС‹Р№ РІС‹РїРѕР»РЅСЏРµС‚ РІС‹РІРѕРґ СЃРїРёСЃРєР° РѕС„РёСЃРѕРІ,
# СЂР°СЃРїРѕР»РѕР¶РµРЅРЅС‹С… РІ SEBEWAING Рё ELECTRON, РєРѕР»РёС‡РµСЃС‚РІРѕ СЂР°Р±РѕС‡РёС… Рё
# СЃСЂРµРґРЅРµР№ Р·Р°СЂРїР»Р°С‚С‹ СЂР°Р±РѕС‡РёС… РІ РєР°Р¶РґРѕРј РѕС„РёСЃРµ

,POSITION
0,C# DEVELOPER
1,AUTOMATION QA ENGINEER
2,JAVA DEVELOPER


In [6]:
pd.read_sql_query(
    """ SELECT OFFICE, count(WORKER_ID), avg(SALARY) as MEAN_SALARY
        FROM job
        WHERE OFFICE_CITY IN ("SEBEWAING", "ELECTRON")
        GROUP BY OFFICE; """,
        con)

# %% [markdown]
# c. РќР°РїРёС€РёС‚Рµ Р·Р°РїСЂРѕСЃ, РєРѕС‚РѕСЂС‹Р№ РІС‹РІРѕРґРёС‚ РЅР°Р·РІР°РЅРёРµ РїСЂРѕС„РµСЃСЃРёР№ (Р±РµР· РїРѕРІС‚РѕСЂРµРЅРёР№),
# РІСЃРµ РїСЂРµРґСЃС‚Р°РІРёС‚РµР»Рё РєРѕС‚РѕСЂРѕР№ РїРѕР»СѓС‡Р°СЋС‚ Р·Р°СЂРїР»Р°С‚Сѓ РІ РїСЂРѕРјРµР¶СѓС‚РєРµ 400-700

,OFFICE,count(WORKER_ID),MEAN_SALARY
0,TIME IS MONEY,7,725.958571


In [7]:
pd.read_sql_query(
    """ SELECT DISTINCT(POSITION)
        FROM job
        WHERE POSITION NOT IN
            (SELECT POSITION
            FROM job
            WHERE SALARY NOT BETWEEN 400 AND 700); """,
        con)

# %% [markdown]
# d. РќР°РїРёС€РёС‚Рµ Р·Р°РїСЂРѕСЃ, РєРѕС‚РѕСЂС‹Р№ РІС‹РІРѕРґРёС‚ СЃСЂРµРґРЅСЋСЋ Р·Р°СЂРїР»Р°С‚Сѓ СЂР°Р±РѕС‚РЅРёРєРѕРІ,
# РєРѕС‚РѕСЂС‹Рµ СЃС‚Р°СЂС€Рµ 30 Р»РµС‚, РіСЂСѓРїРїРёСЂСѓСЏ РїРѕ РїРѕР»Сѓ Рё РІРѕР·СЂР°СЃС‚Сѓ,
# РЅРѕ РїСЂРё РІС‹РІРѕРґРµ СѓРІРµР»РёС‡РёРІР°СЋС‰РёР№ РґР°РЅРЅС‹Рµ Рѕ РІРµР»РёС‡РёРЅРµ Р·Р°СЂРїР»Р°С‚С‹ РЅР° 20%

,POSITION
0,AUTOMATION QA ENGINEER
1,JAVA DEVELOPER


In [8]:
pd.read_sql_query(
    """ SELECT  substr(date(),1,4) - 19||substr(DOB,8,2) AS AGE,
                GENDER,
                AVG(SALARY) * 1.2 AS AVG_SALARY
        FROM workers t1
        INNER JOIN job t2
        ON t1.WORKER_ID = t2.WORKER_ID
        WHERE AGE > 30
        GROUP BY AGE, GENDER; """,
        con)

,AGE,GENDER,AVG_SALARY
0,34,F,1052.370
1,34,M,691.544
2,35,F,1175.004
3,39,F,665.244
